In [1]:
import fileIO
import splice_files
import validate_timestamps
import coordinate_utilities
import time_utilities

from numpy import linspace
from math import ceil

In [2]:
# enter name and directory of input file
input_filename1  = 'workingDir/suunto_gpx.gpx'
input_filename2  = 'workingDir/beep2.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': False,
    'eleflag':  True,
    'hrflag':   False, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [18]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1,skip_metadata=True)
trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2,skip_metadata=True)
trkptlist2 = trkptlist2[0:32]

14638 points imported from workingDir/suunto_gpx.gpx.
81 points imported from workingDir/beep2.gpx.


In [19]:
coordinate_utilities.compute_total_distance(trkptlist2)

821.948497128428

In [20]:
# create list of the % of the distance between each node
ds = coordinate_utilities.distance_per_segment(trkptlist2)
print(ds)

[0, 5.401033252395345, 30.921036205594735, 23.170251538756595, 9.44394807646818, 6.907807726423372, 53.22104010876786, 73.05010501481162, 33.955174796408365, 23.36166931779411, 13.116158214681898, 11.680836967308244, 51.80035629827917, 38.33741197560211, 66.49037755009007, 6.671695598882477, 60.27703201884269, 21.542919293515148, 19.156698333923625, 6.984909668921348, 6.400566532904066, 18.410581071460705, 24.825607918175834, 4.924674866457135, 17.864977716410337, 8.857945192389515, 26.142097208922387, 26.820852298188132, 18.61807572899216, 82.45807717690616, 30.02263019391016, 1.1119492662446386]


Now define how many points need to be added between each node:

In [21]:
trkptlist2_dense = coordinate_utilities.add_interpolated_points(trkptlist2,data_flags2,400)

Original number of points: 32, new number of points: 399.
Original distance: 821.948497128428, new distance: 821.9484971286606.


Interpolated points are successfully added when the distance is preserved but the length has increased.

Generate timestamps for the missing GPX coordinates

Before splicing the files, let's fix the timestamps.

In [38]:
ts1 = {'year': 2023, 'month': 12, 'day': 21, 'hour': 20, 'minute': 19, 'second': 00}
ts2 = {'year': 2023, 'month': 12, 'day': 21, 'hour': 20, 'minute': 25, 'second': 39}
gap_timestamps = time_utilities.generate_timepoints(ts1,ts2)
# print(gap_timestamps)
print(len(gap_timestamps))

400


In [39]:
trkptlist2_dense = time_utilities.add_non_dummy_times(trkptlist2_dense,gap_timestamps)
validate_timestamps.compare_trkpt(trkptlist2_dense)

Matching 400 timestamps to 399 GPS points.
Timestamp start: 2023-12-21T20:19:00
Timestamp end:   2023-12-21T20:25:39          

Fail: 0, Success: 398


In [40]:
# one last check:
for trkpt  in trkptlist2_dense:
    if trkpt.hr == None:
       trkpt.hr = 0 

Export the file:

In [43]:
data_flags3 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
output_filename = "outputDir/garbagio.gpx"
fileIO.export_trackpoints(data_flags3, trkptlist2_dense, output_filename)

Exporting file...
File exported to outputDir/garbagio.gpx
